<a href="https://colab.research.google.com/github/And0k/jupyter-space/blob/main/%D0%9A%D0%BE%D1%80%D0%B6_%D0%90_%D0%9C%D0%98%D0%98_%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%81%D1%8A%D0%B5%D0%B4%D0%BE%D0%B1%D0%BD%D1%8B%D1%85_%D0%B4%D0%B8%D0%BA%D0%BE%D1%80%D0%B0%D1%81%D1%82%D1%83%D1%89%D0%B8%D1%85_%D1%80%D0%B0%D1%81%D1%82%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os

In [2]:
import plotly.express as px  # Library that generates interactive web-based visualizations.

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
# Define dataset and metadata file names and paths
dataset_name = "gverzea/edible-wild-plants"
metadata_file = "edible wild plants metadata.xls"
my_drive_dest = "drive/My Drive/Colab Notebooks/Методы ИИ"

# Extract dataset stem from dataset name
dataset_stem = dataset_name.split('/', 1)[-1]
dataset_path = rf'{my_drive_dest}/{dataset_stem}'

# Загрузка датасета с Kaggle и копирование на Google Диск (при первом запуске)
(you'll need press button to select and upload kaggle.json from you local computer if interaction with Kaggle will be needed)

In [5]:
# Check if metadata file exists, if not, download and extract dataset
metadata_path = f'{dataset_path}/{metadata_file}'
if not os.path.exists(metadata_path):
    dataset_zip = f"{dataset_stem}.zip"  # file name that appears to be downloaded after downloading dataset
    dataset_zip_path = rf'{my_drive_dest}/{dataset_zip}'
    if not os.path.exists(dataset_zip_path):
        my_drive_dest_for_cmd = my_drive_dest.replace(' ', '\ ')

        # Kaggle access file (from my_drive_dest or upload)
        if not os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
            !mkdir ~/.kaggle
            if os.path.exists(rf'{my_drive_dest}/kaggle.json'):
                !cp $my_drive_dest_for_cmd/kaggle.json ~/.kaggle/
            else:
                print("Select kaggle.json to dounload to Google Colab")
                from google.colab import files
                uploaded = files.upload()
                !cp kaggle.json ~/.kaggle/
            !chmod 600 ~/.kaggle/kaggle.json

        # Dataset zip file (autoskipping if exist)
        !kaggle datasets download $dataset_name -p $my_drive_dest_for_cmd

    import zipfile

    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_path)

# # Directory to extract the contents
# extract_path = 'dataset_folder'

# # Create the extraction directory if it doesn't exist
# os.makedirs(extract_path, exist_ok=True)

# # Extract the zip file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# Просмотр состава данных

In [17]:
# Load metadata from Excel file
df_in = pd.read_excel(metadata_path)
df_in.set_index("Name", inplace=True)
print(df_in['# pics'])

Name
Alfalfa                50
Asparagus             100
Blue Vervain           50
Broadleaf Plantain     50
Bull Thistle           50
                     ... 
Wild Bee Balm          50
Wild Black Cherry      50
Wild Grape Vine       100
Wild Leek             100
Wood Sorrel            50
Name: # pics, Length: 62, dtype: int64


# Создание класса Custom Dataset в PyTorch

In [18]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import glob
from sklearn.model_selection import train_test_split

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

False
cpu


# Проверка наличия изображений данных в ожидаемых директориях и отображение сетки изображений

In [20]:
import random
from PIL import Image

# Check if images are available in the expected directories
def check_images_availability(dataset_path):
    subfolders = ['dataset', 'dataset-test', 'dataset-user_images']
    for subfolder in subfolders:
        subfolder_path = os.path.join(dataset_path, subfolder)
        if not os.path.exists(subfolder_path):
            print(f"Directory {subfolder_path} does not exist.")
            return False
        if not os.listdir(subfolder_path):
            print(f"Directory {subfolder_path} is empty.")
            return False
    return True

# Draw a grid of one image for each class
def draw_image_grid(dataset_path):
    classes = os.listdir(os.path.join(dataset_path, 'dataset'))
    num_classes = len(classes)
    fig, axes = plt.subplots(1, num_classes, figsize=(20, 20))
    for i, cls in enumerate(classes):
        class_path = os.path.join(dataset_path, 'dataset', cls)
        image_file = random.choice(os.listdir(class_path))
        image_path = os.path.join(class_path, image_file)
        image = Image.open(image_path)
        axes[i].imshow(image)
        axes[i].set_title(cls)
        axes[i].axis('off')
    plt.show()

# Check images availability and draw grid
if check_images_availability(dataset_path):
    draw_image_grid(dataset_path)
else:
    print("Images are not available in the expected directories.")

Directory drive/My Drive/Colab Notebooks/Методы ИИ/edible-wild-plants/dataset does not exist.
Images are not available in the expected directories.


In [19]:


class EdibleWildPlantsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob.glob(f'{root_dir}/**/*.jpg', recursive=True)
        self.labels = [path.split('/')[-2] for path in self.image_paths]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Предобработка изображений

In [ ]:
from torchvision.transforms import v2

# Define the data transformations
data_transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True),  # Randomly crop and resize images to 224x224 pixels with antialiasing
    v2.RandomHorizontalFlip(),  # Randomly flip images horizontally
    v2.RandomRotation(10),  # Randomly rotate images by 10 degrees
    v2.ToTensor(),  # Convert images to PyTorch tensors
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images with mean and std
])

# Разделение датасета на обучающую и тестовую выборки

In [ ]:
# Split dataset into training and testing sets
dataset = EdibleWildPlantsDataset(root_dir=dataset_path, transform=data_transforms)
train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# Примеры распознавания из папки dataset-user_images

In [ ]:
# Load and display examples from user images dataset
user_images_dataset = EdibleWildPlantsDataset(root_dir=f'{dataset_path}/dataset-user_images', transform=data_transforms)

for i in range(5):
    image, label = user_images_dataset[i]
    plt.imshow(image.permute(1, 2, 0))
    plt.title(label)
    plt.show()